### Feedforward Neural Networks

#### Packages selection 

In [2]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torch.autograd import Variable

### Settings
- device configuration
- hyperparameters definition
- import, Load and exlore data

In [3]:
# device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Hyper-parameters

input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

# MNIST -> datasets

train_dataset = torchvision.datasets.MNIST(root='data',
                                          train=True,
                                          transform=transforms.ToTensor(),
                                          download=True)

test_dataset = torchvision.datasets.MNIST(root='data',
                                         train=False,
                                         transform=transforms.ToTensor())

# dataloader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                         batch_size=batch_size,
                                         shuffle=False)

#### Define the arcitecture of the model such as
- The number of input layers; which is determined by the features of the data
- Number of total hidden layers in the model (iterative)
- Number of hidden units in each layers (iterative)
- The output layer node units is determined by the intended outcome to achieve

In [13]:
# Fully connected Neural Network with one hidden layer

class FedWardNet(nn.Module):
    def __init__ (self, input_size, hidden_size, num_classes):
        super(FedWardNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
            out = self.fc1(x)
            out = self.relu(out)
            out = self.fc2(out)
            return out

#### Loss function and optimizer
- Instantiate the model
- define the specific Loss function to be used either cross entropy, MSELoss, etc
- define the optimization algorithm to be used either SGD, Adam, RMSprop, Momentum etc

In [14]:
models = FedWardNet(input_size, hidden_size, num_classes).to(device)
# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#### Training a model requires the following steps
- Reset all the gradients to zero (0)
- Make a forward pass (make a prediction)
- Calculate the loss
- Perform back propagation
- Update all the parameters (weight and biases)

In [15]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Move tensor to the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = models(images)
        loss = criterion(outputs, labels)
        
        #Backward and optimize/update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss))

Epoch [1/5], Step [100/600], Loss: 2.3266
Epoch [1/5], Step [200/600], Loss: 2.3169
Epoch [1/5], Step [300/600], Loss: 2.3260
Epoch [1/5], Step [400/600], Loss: 2.3056
Epoch [1/5], Step [500/600], Loss: 2.3205
Epoch [1/5], Step [600/600], Loss: 2.3133
Epoch [2/5], Step [100/600], Loss: 2.2946
Epoch [2/5], Step [200/600], Loss: 2.3188
Epoch [2/5], Step [300/600], Loss: 2.3223
Epoch [2/5], Step [400/600], Loss: 2.3020
Epoch [2/5], Step [500/600], Loss: 2.3155
Epoch [2/5], Step [600/600], Loss: 2.3310
Epoch [3/5], Step [100/600], Loss: 2.3320
Epoch [3/5], Step [200/600], Loss: 2.3301
Epoch [3/5], Step [300/600], Loss: 2.3296
Epoch [3/5], Step [400/600], Loss: 2.3159
Epoch [3/5], Step [500/600], Loss: 2.3180
Epoch [3/5], Step [600/600], Loss: 2.3116
Epoch [4/5], Step [100/600], Loss: 2.3184
Epoch [4/5], Step [200/600], Loss: 2.3233
Epoch [4/5], Step [300/600], Loss: 2.3009
Epoch [4/5], Step [400/600], Loss: 2.3272
Epoch [4/5], Step [500/600], Loss: 2.3265
Epoch [4/5], Step [600/600], Loss:

### Testing

In [18]:
# Test the model
# in this phase we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = models(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print('Accuracy of the network on the 10000 test images: {} % '.format(100 * correct / total))
    
# save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Accuracy of the network on the 10000 test images: 7.51 % 
